<a href="https://colab.research.google.com/github/AffanA2003/X-ray-Dental-Detection/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import json  # Import the json module to parse string values into dictionaries
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [4]:
csv_path = 'dataset.csv'
df = pd.read_csv(csv_path)

In [5]:
img_dir = 'images/'
image_paths = []
annotations = []
labels = []

for index, row in df.iterrows():
    img_filename = row['filename']
    
    # Parse the string values into dictionaries
    region_shape_attributes = json.loads(row['region_shape_attributes'])
    region_attributes = json.loads(row['region_attributes'])
    
    annotation = {
        'x': region_shape_attributes['x'],
        'y': region_shape_attributes['y'],
        'width': region_shape_attributes['width'],
        'height': region_shape_attributes['height']
    }
    status = region_attributes['status']
    
    img_path = os.path.join(img_dir, img_filename)
    
    image_paths.append(img_path)
    annotations.append(annotation)
    labels.append(status)

annotations = np.array(annotations)
labels = np.array(labels)
image_paths, annotations, labels = shuffle(image_paths, annotations, labels, random_state=42)

In [6]:
train_image_paths, test_image_paths, train_annotations, test_annotations, train_labels, test_labels = train_test_split(
    image_paths, annotations, labels, test_size=0.2, random_state=42)

def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img = np.array(img) / 255.0
    return img
    
train_labels = np.array(train_labels)
test_labes = np.array(test_labels)
train_annotations = np.array(train_annotations)
test_annotations = np.array(test_annotations)
train_images = np.array([load_and_preprocess_image(img_path) for img_path in train_image_paths])
test_images = np.array([load_and_preprocess_image(img_path) for img_path in test_image_paths])

In [29]:
train_labels = np.array(train_labels)
test_labes = np.array(test_labels)

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

In [8]:

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_split=0.2)


58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/10
6/6 [==============================] - 43s 3s/step - loss: 8.8396 - accuracy: 0.4368 - val_loss: 1.6012 - val_accuracy: 0.7083
Epoch 2/10
6/6 [==============================] - 3s 443ms/step - loss: 1.1977 - accuracy: 0.6158 - val_loss: 0.9365 - val_accuracy: 0.7083
Epoch 3/10
6/6 [==============================] - 3s 467ms/step - loss: 0.9669 - accuracy: 0.6158 - val_loss: 0.8006 - val_accuracy: 0.7083
Epoch 4/10
6/6 [==============================] - 3s 465ms/step - loss: 0.9757 - accuracy: 0.6158 - val_loss: 0.8531 - val_accuracy: 0.7083
Epoch 5/10
6/6 [==============================] - 3s 443ms/step - loss: 0.9479 - accuracy: 0.6158 - val_loss: 0.8973 - val_accuracy: 0.7083
Epoch 6/10
6/6 [==============================] - 3s 464ms/step - loss: 0.9433 - accuracy: 0.6158 - val_loss: 0.8369 - val_accuracy: 0.7083
Epoch 7/10
6/6 [==============================] - 3s 445ms/step - loss: 0.9305 - accuracy: 0.6158

60
60


In [50]:
from sklearn.metrics import accuracy_score, mean_squared_error

# Assuming you have test_annotations and test_labels available

# Calculate the predictions
predictions = model.predict(test_images)

# Convert predictions to class labels (if applicable)


# Calculate the accuracy



# Calculate the mean squared error
mse = mean_squared_error(test_annotations['x'], predictions)



# Print the test loss and accuracy
print("Test Loss (MSE):", mse)
print("Test Accuracy:", accuracy)


2/2 [==============================] - 0s 152ms/step


IndexError: ignored